In [38]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib as mt
import sklearn as sk
import scipy as sc
import statsmodels as st
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

from scipy import stats
from statsmodels import stats
from statsmodels.formula.api import logit, probit, poisson, ols
from sklearn.linear_model import LogisticRegression

# MANIPULATION CHECK

In [39]:
#agent pionowo database
ap = pd.read_excel("C:\\Users\\Justyna\\Python Scripts\\ultimatum2\\data\\agentsTouchPion.xlsx")

In [40]:
ap.head()

,culture,ID,agent,attractiveness,status,masculinity,afect,whatemotion,intesitiy,approachavoid,t_surprising,t_pleasant,t_weak,t_adequate,t_wellsync,t_strange,t_frightening,t_nice
0,Catalan,0,high status,40,70,86,44,none,0,48,2,2,1,3,5,4,2,3
1,Catalan,1,high status,77,75,83,75,none,58,71,4,3,1,3,4,2,1,3
2,Catalan,2,high status,73,69,79,27,disgust,65,34,3,2,3,4,5,3,4,2
3,Catalan,3,high status,63,66,50,68,joy,1,49,2,3,3,3,2,2,1,3
4,Catalan,4,high status,73,73,45,69,joy,18,81,3,4,2,3,4,4,1,2


In [41]:
#labels into numeric, low status = 0, high status = 1 
ap['agentNum'] = ap['agent'].apply(lambda x: 0 if x == "low status" else 1)

#labels into numeric, Catalan culture = 0 , Polish = 1 
ap['cult'] = ap['culture'].apply(lambda x: 0 if x == "Catalan" else 1)

In [42]:
pl = ap[ap["culture"] == "pl"]
cat = ap[ap["culture"] == "cat"]

In [43]:
print(ap.groupby([ "culture", "agent"])[['attractiveness', 'status', 'masculinity']].mean(),
      "                                         ",
      ap.groupby(["culture", "agent"])[['attractiveness', 'status', 'masculinity']].std())

                     attractiveness     status  masculinity
culture agent                                              
Catalan high status       49.000000  73.074074    73.814815
        low status        22.259259  30.629630    75.185185
Polish  high status       50.040000  63.760000    56.560000
        low status        26.320000  30.440000    43.720000                                                                attractiveness     status  masculinity
culture agent                                              
Catalan high status       21.872005   8.818363    14.570880
        low status        20.182995  11.682163    16.895691
Polish  high status       14.650028  13.068027    20.814818
        low status        17.432537  17.173332    18.718351


# descriptive stats for scales

In [44]:
#all = pd.read_excel("C:\\Users\\swidrak\\OneDrive\\python\\ultimatum2\\data\\ultimatum2.xlsx")
all = pd.read_excel("C:\\Users\\Justyna\\Python Scripts\\ultimatum2\\data\\ultimatum2.xlsx")

## Embodiment

### items:
- "Czułem, że wirtualne ciało, które widziałem spoglądając w dół, to moje własne ciało.",
- "Czułem, że wirtualne ciało, które widziałem w lustrze to moje własne ciało.",
- "Czułem, że mam dwa ciała.",
- "Czułem, że ruchy mojego wirtualnego ciała są wywołane moimi własnymi ruchami.",
- "Zachowywałem się w wirtualnym świecie, tak, jak zachwywałbym się w prawdziwej sytuacji."


emb_scale = ["- 3 zdecydowanie się nie zgadzam", "-2","-1","0","1","2", "3 zdecydowanie się zgadzam"]
emb_values = [-3,-2,-1,0,1,2,3]

### Cronbach's alfa ?

### mean, SD x country

In [8]:
#all.head(3)

In [45]:
gex = all[all["round"] == 1]

In [46]:
print(gex.groupby("country")["emb"].describe())

         count      mean       std  min       25%       50%  75%  max
country                                                              
cat       27.0  1.308642  0.934504 -0.5  0.833333  1.333333  2.0  3.0
pl        25.0  0.800000  1.180395 -1.6  0.000000  1.000000  1.4  2.8


### ANOVA

## Social Presence

### items:
- "Czułem, że przebywam w pomieszczeniu w obecności innej osoby.",
- "Czułem, że ta osoba na mnie patrzy i jest świadoma mojej obecności.",
- "Myśl o tym, że ta osoba nie jest prawdziwa, często przebiegała przez moją głowę.",
- "Ta postać sprawiała dla mnie wrażenie żywej, czującej i świadomej.",
- "Odebrałem tę postać jako tylko komputerowy obraz, a nie żywą osobę."

soc_pres_scale = ['zdecydowanie się nie zgadzam','','','','zdecydowanie się zgadzam']

values: [1,2,3,4,5]

### Cronbach's alfa

### mean, SD x country

In [11]:
print(gex.groupby("country")[["socPres"]].describe())

        socPres                                             
          count      mean       std  min  25%  50%  75%  max
country                                                     
cat        27.0  2.844444  0.812088  1.4  2.3  2.8  3.3  4.2
pl         25.0  2.688000  0.828814  1.2  2.2  2.6  3.2  4.2


### ANOVA

## ATG

### items:
- "Męskie pary homoseksualne powinny mieć takie same prawa do adoptowania dzieci, jakie mają pary heteroseksualne.",
- "Myślę, że męski homoseksualizm jest obrzydliwy.",
- "Mężczyzna - homoseksualista nie powinien mieć prawa uczenia w szkole.",
- "Męski homoseksualizm jest perwersją.",
- "Męski homoseksualizm jest naturalną ekspresją seksualności u mężczyzny.",
- "Jeśli mężczyzna ma homoseksualne odczucia, powinien zrobić wszystko, co może, by je powstrzymać.",
- "Nie byłbym zbyt poruszony, gdybym się dowiedział, że mój syn jest homoseksualistą.",
- "Sex dwóch mężczyzn jest czymś po prostu złym.",
- "Idea męskiego homoseksualnego małżeństwa jest dla mnie żałośnie śmieszna.",
- "Męski homoseksualizm jest tylko innym stylem życia, który nie powinien być potępiany."


atg_scale = ["zdecydowanie się nie zgadzam","","","","","","","","zdecydowanie się zgadzam"]

values: [1,2,3,4,5,6,7,8,9]

### Cronbach's alfa

### mean, SD x country

In [12]:
print(gex.groupby("country")[["atg"]].describe())

          atg                                               
        count      mean       std  min   25%  50%   75%  max
country                                                     
cat      27.0  2.174074  1.137336  1.0  1.25  1.8  2.95  5.8
pl       25.0  4.888000  2.259668  1.1  3.40  4.8  6.70  9.0


### ANOVA

## ISS

### items:
- "Wysoki status społeczny jest dla mnie ważny.",
- "Wybrałem moją ścieżkę edukacji i/lub zawód głównie ze względu na ich prestiżowy charakter.",
- "Osobom postawionym wyżej ode mnie okazuję szczególny szacunek.",
- "Każdgo traktuję tak samo, niezależnie od tego, czy stoi na drabinie społecznej wyżej czy niżej niż ja."

stat_scale = ['zdecydowanie się nie zgadzam','','','','zdecydowanie się zgadzam']

### Cronbach's alfa

### mean, SD x country 

In [13]:
print(gex.groupby("country")[["stat"]].describe())

         stat                                               
        count      mean       std  min  25%   50%  75%   max
country                                                     
cat      27.0  2.407407  0.871968  0.0  2.0  2.25  3.0  3.75
pl       25.0  2.600000  0.721688  1.0  2.0  2.75  3.0  4.00


### ANOVA

# HYPOTHESES VERIFICATION

In [47]:
all = all[all["cut"] == "keep"]
all = all[all["cut2"] == "keep"]

In [48]:
#new columns
all.loc[:,'name'] = all.loc[:,'agent'].apply(lambda x: "low status" if x == 0 else "high status")

all['touchN'] = all.loc[:,('touch')].apply(lambda x: 0 if x == 0.0 else 1)

all['cult'] = all.loc[:,('country')].apply(lambda x: 1 if x == "pl" else 0)

all['offerParC'] = all.loc[:,'offerPar'].apply(lambda x: "unfair" if x < 36 else ("fair" if x > 36 and x < 46 else "generous"))

In [49]:
pro = all[all["parPro"] == 1]
res = all[all["parPro"] == 0]

pl = all[all["country"] == "pl"]
ca = all[all["country"] == "cat"]

plpro = pl[pl["parPro"] == 1]
capro = ca[ca["parPro"] == 1]

plres = pl[pl["parPro"] == 0]
cares = ca[ca["parPro"] == 0]

resyes = res[res["accepted"] == 1]
resno = res[res["accepted"] == 0]

## offers acceptance

### H1
### People will play ultimatum game with embodied agents “mindlessly”, treating them as if they were humans. 
#### This implies they will make rather fair offers (60/40, ref.) because of altruism and/or fear of being punished for too greedy offers.

In [74]:
#acceptance rate by country
res.groupby(["cult", "offerParC"])["accepted"].describe()

count      mean       std  min  25%  50%  75%  max
cult offerParC                                                    
0    fair       162.0  0.382716  0.487557  0.0  0.0  0.0  1.0  1.0
     generous    54.0  0.944444  0.231212  0.0  1.0  1.0  1.0  1.0
     unfair     108.0  0.083333  0.277674  0.0  0.0  0.0  0.0  1.0
1    fair       150.0  0.400000  0.491539  0.0  0.0  0.0  1.0  1.0
     generous    50.0  0.800000  0.404061  0.0  1.0  1.0  1.0  1.0
     unfair     100.0  0.140000  0.348735  0.0  0.0  0.0  0.0  1.0

In [68]:
#acceptance by category, agent, and culture
res.groupby(["country", "offerParC","name"])["accepted"].describe()

count      mean       std  min  25%  50%  75%  \
country offerParC name                                                         
cat     fair      high status   81.0  0.444444  0.500000  0.0  0.0  0.0  1.0   
                  low status    81.0  0.320988  0.469765  0.0  0.0  0.0  1.0   
        generous  high status   27.0  0.888889  0.320256  0.0  1.0  1.0  1.0   
                  low status    27.0  1.000000  0.000000  1.0  1.0  1.0  1.0   
        unfair    high status   54.0  0.074074  0.264351  0.0  0.0  0.0  0.0   
                  low status    54.0  0.092593  0.292582  0.0  0.0  0.0  0.0   
pl      fair      high status   75.0  0.466667  0.502247  0.0  0.0  0.0  1.0   
                  low status    75.0  0.333333  0.474579  0.0  0.0  0.0  1.0   
        generous  high status   25.0  0.760000  0.435890  0.0  1.0  1.0  1.0   
                  low status    25.0  0.840000  0.374166  0.0  1.0  1.0  1.0   
        unfair    high status   50.0  0.160000  0.370328  0.0  0.0  0.0  0.0   
                  low status    50.0  0.120000  0.328261  0.0  0.0  0.0  0.0   

                               max  
country offerParC name              
cat     fair      high status  1.0  
                  low status   1.0  
        generous  high status  1.0  
                  low status   1.0  
        unfair    high status  1.0  
                  low status   1.0  
pl      fair      high status  1.0  
                  low status   1.0  
        generous  high status  1.0  
                  low status   1.0  
        unfair    high status  1.0  
                  low status   1.0

## made offers

In [50]:
#cutting into categories (into defined number of even categories or with defined bins)

bins = pd.IntervalIndex.from_tuples(((0,10), (10,20), (20,30), (30,40), 
                                     (40,50), (50,60), (60,70), (70,80),
                                    (80,90), (90,100)))

pro.loc[:,"offerAgC"] = pd.cut(x = pro.loc[:,"offerAg"], bins = bins, labels = [1,2,3,4,5,6,7,8,9,10])

#pro.loc[:,("offerAgC", "offerAg")]

C:\Users\Justyna\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Justyna\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [73]:
# mean value of offers for agents by country
pro.groupby(["agent", "country"])["offerAg"].describe()

count       mean        std   min   25%   50%   75%   max
agent country                                                           
0.0   cat       81.0  45.703704   8.078435  20.0  40.0  47.0  50.0  85.0
      pl        75.0  42.920000   8.957497   0.0  40.0  45.0  50.0  55.0
1.0   cat       54.0  44.240741  10.013077   3.0  40.0  46.5  50.0  65.0
      pl        50.0  44.140000  11.596991   1.0  40.0  45.0  50.0  90.0

In [52]:
pro.groupby(["offerAgC", "country"])["offerAg"].describe()

count       mean       std   min    25%   50%    75%   max
offerAgC country                                                            
(0, 10]  cat        2.0   4.000000  1.414214   3.0   3.50   4.0   4.50   5.0
         pl         2.0   5.500000  6.363961   1.0   3.25   5.5   7.75  10.0
(10, 20] cat        1.0  20.000000       NaN  20.0  20.00  20.0  20.00  20.0
         pl         1.0  18.000000       NaN  18.0  18.00  18.0  18.00  18.0
(20, 30] cat        3.0  30.000000  0.000000  30.0  30.00  30.0  30.00  30.0
         pl         5.0  26.800000  2.489980  25.0  25.00  25.0  29.00  30.0
(30, 40] cat       34.0  38.411765  2.284568  34.0  35.25  40.0  40.00  40.0
         pl        34.0  38.117647  2.446577  32.0  36.25  39.5  40.00  40.0
(40, 50] cat       84.0  47.869048  2.701092  41.0  45.00  50.0  50.00  50.0
         pl        77.0  47.532468  2.736615  41.0  45.00  48.0  50.00  50.0
(50, 60] cat        9.0  55.111111  3.100179  52.0  52.00  55.0  55.00  60.0
         pl         4.0  54.250000  4.272002  51.0  51.00  53.0  56.25  60.0
(60, 70] cat        1.0  65.000000       NaN  65.0  65.00  65.0  65.00  65.0
(80, 90] cat        1.0  85.000000       NaN  85.0  85.00  85.0  85.00  85.0
         pl         1.0  90.000000       NaN  90.0  90.00  90.0  90.00  90.0

## compliance (GEE model) 

In [22]:
data = res
fam = sm.families.Binomial()

gor = sm.cov_struct.Autoregressive()
mod = smf.gee("accepted ~ touch*agent*cult + cult*Zatg + cult*Zstat + Zemb + ZsocPres",
              "ID", data, cov_struct=gor, family=fam)

#summ = sm.genmod.cov_struct.GlobalOddsRatio("nominal")

results = mod.fit()
print(results.summary())
print(gor.summary())
#print(summ.summary())

## Wald chi2 i p = .000 (wyniki wciaz nieco inne niz w SPSS ale juz blisko)
#print(resu.wald_test("touch"))

## tutaj cala tabela razem z uwzglednieniem interakcji
print(results.wald_test_terms())

                               GEE Regression Results                              
Dep. Variable:                    accepted   No. Observations:                  624
Model:                                 GEE   No. clusters:                       52
Method:                        Generalized   Min. cluster size:                  12
                      Estimating Equations   Max. cluster size:                  12
Family:                           Binomial   Mean cluster size:                12.0
Dependence structure:       Autoregressive   Num. iterations:                     9
Date:                     Mon, 27 Aug 2018   Scale:                           1.000
Covariance type:                    robust   Time:                         19:11:13
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.2654      0.291     -4.350      0.000      -1.836  

## generosity (Linear Mixed Effects fit in GEE so far) 

In [67]:
ind = sm.cov_struct.Autoregressive()
fml = "offerAg ~ agent*cult + Zatg*cult + Zstat*cult + Zemb + ZsocPres"
mod_gee = sm.GEE.from_formula(fml, groups=pro["ID"], cov_struct=ind, data=pro)
mod_gee = mod_gee.fit()

print(mod_gee.summary())
print(mod_gee.cov_struct.summary())
print(mod_gee.wald_test_terms())
print(ind.summary())

# http://nbviewer.jupyter.org/urls/umich.box.com/shared/static/lc6uf6dmabmitjbup3yt.ipynb

                               GEE Regression Results                              
Dep. Variable:                     offerAg   No. Observations:                  260
Model:                                 GEE   No. clusters:                       52
Method:                        Generalized   Min. cluster size:                   5
                      Estimating Equations   Max. cluster size:                   5
Family:                           Gaussian   Mean cluster size:                 5.0
Dependence structure:       Autoregressive   Num. iterations:                     9
Date:                     Mon, 27 Aug 2018   Scale:                          76.870
Covariance type:                    robust   Time:                         20:01:24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     44.8732      1.619     27.718      0.000      41.700      48.046
agent  

# PLOTS

In [ ]:
# setting palette
pal = ["#01A183", "#696969", "#4Cb5F5"]
sns.palplot(sns.color_palette(pal))

In [ ]:
sns.set(font_scale=1.3)

g = plt.figure(figsize=(8,6))#, frameon=False)


g = sns.catplot(x="agent", y="status", col="culture", data=ap, saturation=1.3,
                kind="boxen", ci="sd", aspect=0.6, palette = pal, legend = True, 
                legend_out = True, margin_titles=False, height=5, sharex=False, sharey=True)
g.set_axis_labels("agent", "Assessed status").set_xticklabels()


ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.yticks([0, 25, 50, 75,100], fontsize=14)
plt.ylim(0, 120)




g.savefig('status.png', transparent=True, figsize=(7,6), dpi = 300)

In [ ]:
sns.set(font_scale=1.3)

g = plt.figure(figsize=(8,6))#, frameon=False)


g = sns.catplot(x="agent", y="masculinity", col="culture", data=ap, saturation=1.3,
                kind="boxen", ci="sd", aspect=0.6, palette = pal, legend = True, 
                legend_out = True, margin_titles=False, height=5, sharex=False, sharey=True)
g.set_axis_labels("agent", "Assessed masculinity").set_xticklabels()


ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.yticks([0, 25, 50, 75,100], fontsize=14)
plt.ylim(0, 120)



g.savefig('masculinity.png', transparent=True, figsize=(7,6), dpi = 300)

In [ ]:
sns.set(font_scale=1.3)

g = plt.figure(figsize=(8,6))#, frameon=False)


g = sns.catplot(x="agent", y="attractiveness", col="culture", data=ap, saturation=1.3,
                kind="boxen", ci="sd", aspect=0.6, palette = pal, legend = True, 
                legend_out = True, margin_titles=False, height=5, sharex=False, sharey=True)
g.set_axis_labels("agent", "Assessed attractiveness").set_xticklabels()


ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.yticks([0, 25, 50, 75,100], fontsize=14)
plt.ylim(0, 120)


g.savefig('attractiveness.png', transparent=True, figsize=(7,6), dpi = 300)

In [ ]:
mt.style.use('seaborn-paper')

f = plt.figure(figsize=(9,6))

plt.hist(x = "offerPar", data = resyes, 
         bins = 25, 
         alpha = 0.85, 
         color = "#01A183") #green

plt.hist(x = "offerPar", data = resno, 
         bins = 25, 
         alpha = 0.6,
         color = "#696969")  #grey

plt.plot([50,50], [0,72], color = "red")

ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.xlabel('Value offered to the participant', fontsize=16)
plt.ylabel('Frequency', fontsize = 16)
plt.xticks(fontsize = 16)
plt.yticks([0, 25, 50, 75,100], fontsize=16)
#plt.ylim(-5, 100)
#plt.xlim(-3, 2.5)

#categories
plt.text(x = 30, y = 74, horizontalalignment = 'center', s = "UNFAIR", fontsize = 15)
plt.text(x = 42, y = 74, horizontalalignment = 'center', s = "FAIR", fontsize = 15)
plt.text(x = 48, y = 74, horizontalalignment = 'left', s = "GENEROUS", fontsize = 15)

#legend
plt.legend(('50/50','Offers accepted', 'Offers rejected'), 
           loc=[0.05, 0.8], fontsize=16)

f.savefig('acceptedReject.png',  transparent=True, figsize=(7,6), dpi = 300)

In [ ]:
plt.figure(figsize=(9,6), frameon=False)

g = sns.lmplot(x = "Zemb", 
          y = "offerAg", 
          hue = "country", 
          data = pro, 
          sharex = True,
          sharey = True,
          legend_out=True,
          legend = False,
          scatter = True, 
          fit_reg = True, 
          logistic = False, 
          markers=["+", "*"],
          palette = pal,
          y_jitter = True,
          aspect=1.2) 

    
    
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.xlabel('Embodiment (standardized)', fontsize=14)
plt.ylabel('N coins offered to the agent', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks([0, 25, 50, 75,100], fontsize=14)

#legend
plt.legend(["Catalan", "Polish"], loc="best", fontsize=14)

#g.savefig('linearReg.png', transparent=True, figsize=(9,6), dpi = 300)

### alternatively, after some improvements maybe this:

In [ ]:
plt.figure(figsize=(9,6), frameon=False)

g = sns.lmplot(x = "Zstat", 
          y = "offerAg", 
          col = "country", 
          hue= "agent",
          data = pro, 
          sharex = True,
          sharey = True,
          legend_out=True,
          legend = False,
          scatter = True, 
          fit_reg = True, 
          logistic = False, 
          markers=["+", "*"],
          palette = pal,
          y_jitter = True,
          aspect=1.2)

    
    
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Only show ticks on the left and bottom spines
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.xlabel('Importance of social status (standardized)', fontsize=14)
plt.ylabel('N coins offered to the agent', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks([0, 25, 50, 75,100], fontsize=14)

#legend
#plt.legend(["Catalan", "Polish"], loc="best", fontsize=14)

#g.savefig('linearReg.png', transparent=True, figsize=(9,6), dpi = 300)